Installing Libraries and setting parameters

In [ ]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 32
block_size = 128
max_iter = 2000
learning_rate = 4e-4
eval_iters = 10
n_embd = 384
n_head = 8 
n_layer = 1
drop_out = 0.2
n_layer = 8

print(device)

Load the Shakespeare literature

In [ ]:
chars = ""
with open("shakespeare.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

Tokenizing

In [ ]:


string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
a = 'todo'
e = encode(a)
d = decode(e)


Ingest text and split the data

In [ ]:
data = torch.tensor(encode(text),dtype = torch.long)
split = int(0.8 * len(data))
train_data = data[:split]
val_data = data[split:]

Some functions

In [ ]:
#given the type of data (train or val), return n blocks of random data
def get_batch(type_of_data):
    data = train_data if type_of_data == 'train' else val_data
    start = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in start])
    y = torch.stack([data[i+1:i+block_size+1] for i in start])
    x, y = x.to(device), y.to(device)
    return x, y   

@torch.no_grad()
def estimate_loss():
    output = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        output[split] = losses.mean()
    model.train()
    return output

Head Class

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        #x is of shape (batch_size, time_step, channel)
        #we want to do self attention on the time_step dimension
        #so we permute the time_step and channel dimension
        #and then reshape it to (batch_size * time_step, channel)


        B, T, C = x.shape
        k = self.key(x)     #(B, T, head_size)
        q = self.query(x)   #(B, T, head_size)
        wei = q @ k.transpose(-2,-1) * k.shape[-1] ** -0.5  #(B, T, head_size) @ (B, head_size, T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1) #(B, T, T)
        v = self.value(x)   #(B, T, head_size)
        out = wei @ v
        return out 

MultiHead Attention Class

In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(drop_out)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

Class FeedFordaward

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(drop_out)
        )

    def forward(self, x):
        return self.net(x)

Class Block

In [ ]:
class Block(nn.Module):
    #Transformer block
    def __init__(self, n_embd,n_head):
        #inherits from nn.Module
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiheadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x) 
        x = self.ln2(x + y)
        return x

Class GPT

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, std = 0.02, mean = 0.0)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, std = 0.02, mean = 0.0)
    
    def forward(self, index, targets=None):
        B, T = index.shape


        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


model = GPTLanguageModel(vocab_size)
m = model.to(device)


create an optimizer and iterate over for training the GPT

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
for iter in range(max_iter):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print('iter', iter, 'train loss', losses['train'], 'val loss', losses['val'])
    xb, yb = get_batch('train')
    #evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()


In [ ]:
prompt = str('Hello! Can you see me?')
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)